## Qdrant

- Vector Database
- Open Source
- An alternative to Pinecone database
- Managed Services

www.qdrant.tech

### Setup
- Setup 1GB Free cluster in cloud service
- Vector database persistent in time
- Database available from a URL
- Data available via simple APIs

In [ ]:
# https://87cf3a23-f1db-424a-9d5f-da212d4074aa.us-east4-0.gcp.cloud.qdrant.io

In [1]:
# api_key = ''

In [ ]:
# Cluster > Collection > Vector Store > Point or Vector

In [ ]:
# Vector is a numerical representation of your text

In [ ]:
!pip install qdrant_client openai tiktoken langchain

In [3]:
import os
import qdrant_client

from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings

In [4]:
# # Create a qdrant client

# os.environ['QDRANT_HOST'] = ''
# os.environ['QDRANT_API_KEY'] = ''
os.environ['QDRANT_HOST']


'http://qdrant:6333'

In [5]:
client = qdrant_client.QdrantClient(
    os.getenv('QDRANT_HOST'),
    #api_key = os.getenv('QDRANT_API_KEY')
)

In [6]:
from qdrant_client.http import models

In [13]:
# create a collection (A database with vectors)
# name, size or dimension, distance calculations = COSINE

os.environ["QDRANT_COLLECTION_NAME"] = 'collection1'

vector_config = models.VectorParams(
    size = 1536, # same as openAI
    distance = models.Distance.COSINE
)

client.recreate_collection(
    collection_name = os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config = vector_config
)

True

In [15]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='collection1')])

In [ ]:
# Create a vector store to store the documents

In [ ]:
os.environ["OPENAI_API_KEY"]

In [17]:
# Qdrant client - client, collection name, embeddings

#os.environ["OPENAI_API_KEY"] = ""

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client = client,
    collection_name = os.getenv("QDRANT_COLLECTION_NAME"),
    embeddings = embeddings
)

In [ ]:
# Add a document to vector store

In [ ]:
# long documents > split the document into chunks > 1000 characters

In [ ]:
with open('tsla_news.txt') as f:
  raw_text = f.read()
  print(raw_text)

As a company, Tesla (NASDAQ: TSLA) has been at the center of investors’ attention for many years. On the one hand, it represents the world’s most recognizable EV maker, piloted by one of the world’s most recognized billionaires. 

On the other hand, many have assessed that the firm is overvalued and priced as a big tech company and not a car maker – with Musk himself claiming the former on at least one occasion.

No matter the stock market sentiment toward the EV maker, the numbers paint a decisive yet curious picture. Throughout 2020, Tesla delivered approximately 499 million vehicles and made $31.5 billion in revenue. In the first three quarters of 2023, the company delivered more than 1.3 million vehicles and already accrued more than $72 billion in revenue.

Tesla’s earnings per share and profits have similarly vastly increased between 2020 and 2023 – $0.24 to $3.14 and $782 million to $11 billion, respectively. Despite the indisputable growth, the price of the EV maker’s shares is

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

def get_chunks(text):
  text_splitter = CharacterTextSplitter(
      separator = "\n",
      chunk_size = 200,
      chunk_overlap = 40,
      length_function = len
  )

  chunks = text_splitter.split_text(text)

  return chunks


In [ ]:
texts = get_chunks(raw_text)
print(texts)

['As a company, Tesla (NASDAQ: TSLA) has been at the center of investors’ attention for many years. On the one hand, it represents the world’s most recognizable EV maker, piloted by one of the world’s most recognized billionaires.', 'On the other hand, many have assessed that the firm is overvalued and priced as a big tech company and not a car maker – with Musk himself claiming the former on at least one occasion.', 'No matter the stock market sentiment toward the EV maker, the numbers paint a decisive yet curious picture. Throughout 2020, Tesla delivered approximately 499 million vehicles and made $31.5 billion in revenue. In the first three quarters of 2023, the company delivered more than 1.3 million vehicles and already accrued more than $72 billion in revenue.', 'Tesla’s earnings per share and profits have similarly vastly increased between 2020 and 2023 – $0.24 to $3.14 and $782 million to $11 billion, respectively. Despite the indisputable growth, the price of the EV maker’s sh

In [ ]:
len(texts)

22

In [ ]:
vector_store.add_texts(texts)

['f3467c1746f54fe08692bf4f1fb0c6bf',
 '4fadc267f690402abe51e78e4f1f5d7a',
 'b0315c3d2b9e4e4a8c2bbfa775ae9a22',
 '40523d09495d434589fbab1e0b9a2270',
 '2cdaef6a9c9a4ceea1c64c88326ff2a5',
 'e0d70bc828aa4146986a1e0938f822ee',
 'aed6b24f64d2421eaf5984b70ab12f34',
 '873449f3615f447d97e9eb820730f083',
 '25eeb4dc75cc4917be7858320ac843c6',
 '304608e5249c4fb69301429498faf31b',
 'de893e2086924a75ab77e3b9519e38e6',
 'c2506853bdf64b98a03119bce04af441',
 '3a18ac67cd9b43458f0a8038170567de',
 '63544aad3be94c0b8eb478beccc82480',
 'dd6c7a960cbf473aba9c1d85e275e7d1',
 '09ec006b982340d68fac3c2384183d08',
 'bd7fbbc4823d4a58a25307de120d6873',
 '2cce050901004d96951b60efbca24b50',
 'afbbf791be6b41a2af40162448d9cb35',
 'f41c7f358ef44d73ac04833434eb1b3a',
 'fda1630175604b6788e2fee618ad44c8',
 '6dce0b27d9374d8b8f1cd4380f11848c']

In [ ]:
# query the data

In [ ]:
# retriever

from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.schema import retriever


In [ ]:
qa = RetrievalQA.from_chain_type(
    llm = OpenAI(),
    chain_type = "stuff",
    retriever = vector_store.as_retriever()
)

In [ ]:
query = "What was the tesla earning price"
response = qa.run(query)
print(response)

 Tesla's earnings per share in 2023 was $3.14.


In [ ]:
query = "Give me the top 5 main key points of the news"
response = qa.run(query)
print(response)

 1. Tesla has experienced events that have impacted its stock price, but they may not be more frequent than other manufacturers. 
2. Tesla recently made a breakthrough in the manufacturing process. 
3. Tesla's Berlin plant is likely to begin the production of a new €25,000 model. 
4. Tesla is the world's most recognizable EV maker. 
5. Tesla is piloted by one of the world's most recognized billionaires.
